In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import densenet as dn
import torch
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import os
import torchvision.datasets as dset

In [ ]:
#데이터셋 및 모델저장 경로 지정
dataset_path = "/home/ycs/Downloads/dataset"
model_weight_save_path = "/home/ycs/Downloads/save/"
num_classes = 400

batch_size = 100
num_workers = 3
learning_rate = 0.0001
layers = 100
epoch = 100

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 훈련 및 테스트 데이터셋 경로 로드
traindir = os.path.join(dataset_path, '/home/ycs/Downloads/dataset/train')
testdir = os.path.join(dataset_path, '/home/ycs/Downloads/dataset/train')

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
model = dn.DenseNet3(layers, 400, growth_rate=12, reduction=0.5, bottleneck=True, dropRate=0.0)
#400=클래스수 위에서 입력한 값과 동일하게 바꿔줘야함
#import densenet as dn 을 통해 사전준비된 densenet layer 를 import

In [ ]:
model = model.cuda()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [ ]:
def train(train_loader,model,criterion,optimizer,epoch):
    model.train()
    for i, (input,target) in enumerate(train_loader):
        target = target.to(device)
        input = input.to(device)
        
        output = model(input)
        loss = criterion(output,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i%10 == 0): #10 step 마다 loss값 출력
            print("loss in epoch %d , step %d : %f" % (epoch, i,loss.data))

In [ ]:
def test(test_loader,model,criterion,epoch):
    model.eval()
    
    correct = 0
    
    
    for i, (input,target) in enumerate(test_loader):
        target = target.to(device)
        input = input.to(device)
        
        output = model(input)
        loss = criterion(output,target)
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().float().sum()
    
    print("Accuracy in epoch %d : %f" % (epoch,100.0*correct/len(test_loader.dataset)))

In [ ]:
def adjust_lr(optimizer, epoch, learning_rate): #epoch 10 증가시 lr 값 변화 (변화없음=1)
    if epoch==10 :
        learning_rate*=1
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate

In [ ]:
for epoch in range(0,100):
    adjust_lr(optimizer,epoch,learning_rate)
    train(train_loader,model,criterion,optimizer,epoch)
    test(test_loader,model,criterion,epoch)
    if epoch % 1 == 0: #1회당 한번저장
        torch.save(model.state_dict(), model_weight_save_path + 'model.pt', _use_new_zipfile_serialization=False)
